In [52]:
import pandas as pd
import time
import copy as cp
import numpy as np
import sys

#upit_path = '../minelib_inputs/newman1.upit'
#prec_path = '../minelib_inputs/newman1.prec'

upit_path = '../minelib_inputs/kd.upit'
prec_path = '../minelib_inputs/kd.prec'

In [53]:
lineas = []
w = {}  # lista de pesos
# recuperar los pesos de cada bloque
with open(upit_path, 'r') as file:
    for line in file:
        line_list = line.split()
        lineas.append(line_list)
lineas = cp.copy(lineas[4:])
for nodo_id,linea in enumerate(lineas):
    w[nodo_id] = float(linea[1])
nblocks = len(w)

In [54]:
w

{0: -12285.0,
 1: -12285.0,
 2: -12285.0,
 3: -12285.0,
 4: -12285.0,
 5: -12285.0,
 6: -12285.0,
 7: -12285.0,
 8: -12285.0,
 9: -9000.0,
 10: -12285.0,
 11: -9000.0,
 12: -12285.0,
 13: -12285.0,
 14: -9000.0,
 15: -9000.0,
 16: -12285.0,
 17: -12285.0,
 18: -12285.0,
 19: -9000.0,
 20: -12285.0,
 21: -12285.0,
 22: -12285.0,
 23: -9000.0,
 24: -9000.0,
 25: -12285.0,
 26: -12285.0,
 27: -9000.0,
 28: -9000.0,
 29: -12285.0,
 30: -12285.0,
 31: -12285.0,
 32: -9000.0,
 33: -12285.0,
 34: -12285.0,
 35: -12285.0,
 36: -9000.0,
 37: -12285.0,
 38: -12285.0,
 39: -9000.0,
 40: -12285.0,
 41: -12285.0,
 42: -12285.0,
 43: -12285.0,
 44: -12285.0,
 45: -9000.0,
 46: -12285.0,
 47: -9000.0,
 48: -12285.0,
 49: -9000.0,
 50: -12285.0,
 51: -9000.0,
 52: -12285.0,
 53: -9000.0,
 54: -12285.0,
 55: -12285.0,
 56: -12285.0,
 57: -12285.0,
 58: -12285.0,
 59: -12285.0,
 60: -12285.0,
 61: -12285.0,
 62: -12285.0,
 63: -12285.0,
 64: -12285.0,
 65: -12285.0,
 66: -12285.0,
 67: -12285.0,
 68: -1

In [55]:
# recuperar las precedencias para cada bloque
S = {}  # lista de precedencias inmediatas
with open(prec_path) as file:
    for nodo_id,line in enumerate(file):
        S[nodo_id] = []
        line_list = line.split()
        if not line_list[1] == '0':
            S[nodo_id].extend(list(map(int, cp.copy(line_list[2:]))))            

In [25]:
S

{0: [],
 1: [2, 9],
 2: [3, 10],
 3: [4, 11],
 4: [5],
 5: [0, 6],
 6: [],
 7: [8, 11, 19],
 8: [],
 9: [3, 7, 10, 14, 20],
 10: [4, 8, 11, 15, 21],
 11: [5, 16],
 12: [9, 13],
 13: [10, 14, 22],
 14: [11, 15, 23],
 15: [16],
 16: [6, 17],
 17: [],
 18: [8, 19, 21],
 19: [],
 20: [11, 19, 21, 23],
 21: [],
 22: [15, 21, 23, 25],
 23: [16],
 24: [23, 25, 27, 39],
 25: [],
 26: [25, 27, 40],
 27: [28],
 28: [],
 29: [30],
 30: [],
 31: [38],
 32: [33, 51],
 33: [34, 52],
 34: [35, 53],
 35: [36, 54],
 36: [31, 37],
 37: [38],
 38: [],
 39: [],
 40: [],
 41: [28, 44],
 42: [41, 43, 60],
 43: [44],
 44: [],
 45: [49],
 46: [47, 51, 73],
 47: [45, 48, 52],
 48: [49, 53],
 49: [54],
 50: [33, 47, 51, 75],
 51: [34, 48, 52, 76],
 52: [35, 49, 53, 77],
 53: [36, 54],
 54: [37, 55],
 55: [],
 56: [],
 57: [41, 58, 60],
 58: [],
 59: [43, 58, 60, 62],
 60: [44, 63],
 61: [60, 62],
 62: [63],
 63: [],
 64: [65, 71, 87],
 65: [66, 72],
 66: [73],
 67: [68, 88],
 68: [69, 89],
 69: [64, 70, 90],
 7

In [ ]:
def test_opt(vertices):
    for vertice in vertices:
        for vecino in S[vertice]:
            if vecino not in vertices:
                return vertice, vecino
    return True


def encontrar_camino(origen, destino, arcosArbol):
    antecesor = {}
    visitados = [origen]
    cola = [origen]
    while cola:
        neighbors_nodo = []
        nodo = cola.pop()
        for tail, head in arcosArbol:
            if nodo in (tail, head):
                if not head == nodo:
                    neighbors_nodo.append((head, 'head'))
                else:
                    neighbors_nodo.append((tail, 'tail'))
        cola.extend([vecino for w, label in neighbors_nodo if w not in visitados])
        for vecino, label in neighbors_nodo: 
            if vecino not in visitados:
                antecesor[vecino] = (nodo, label)
                visitados.append(vecino)
        if destino in visitados:
            break
    hoja = cp.copy(destino)
    padre, label_hoja = antecesor[hoja]
    if label_hoja == 'head':
        aristas_camino = [(padre, hoja)]
    else:
        aristas_camino = [(hoja, padre)]
    while not padre == 'vo':
        hoja = padre
        padre, label_hoja = antecesor[hoja]
        if label_hoja == 'head':
            aristas_camino.append((padre, hoja))
        else:
            aristas_camino.append((hoja,padre))
    return aristas_camino

def obtener_rama(raiz, arcosArbol, nodos_fuertes):
    vecinos = []
    for tail, head in arcosArbol:
        if raiz in (tail, head):
            if (not head == 'vo') and head not in nodos_fuertes:
                vecinos.append(head)
            elif (not tail == 'vo') and tail not in nodos_fuertes:
                vecinos.append(tail)
    if vecinos:
        for vecino in vecinos:
            if vecino not in nodos_fuertes:
                nodos_fuertes.append(vecino)
            obtener_rama(vecino, arcosArbol, nodos_fuertes)

In [ ]:
# agregar nodo vo artificial
S['vo'] = []
for nodo_id in range(nblocks):
    S['vo'].append(nodo_id)
T = [('vo', nodo) for nodo in S['vo']]  # incializar arbol
etiquetas = {}  # crear diccionario de etiquetas 
for nodo in S['vo']:  
    etiquetas[('vo', nodo)] = ['+', w[nodo]]
Y = []  # crear lista de nodos fuertes
for nodo_id in range(nblocks):
    if w[nodo_id] > 0:
        Y.append(nodo_id)
init_time = time.time()
while True:
    if test_opt(Y) is True:  # verificar optimalidad
        break
    (vk, vl) = test_opt(Y)  # si no se tiene, actualizar arbol: STEP 3
    aristas_vo_vk_camino = encontrar_camino('vo', vk, T)  # encontrar caminos a extremos de ñas aristas
    vm = cp.copy(aristas_vo_vk_camino[0][1])
    aristas_vo_vk_camino.reverse()  # reverse para que el nombre tenga sentido
    aristas_vl_vo_camino = encontrar_camino('vo', vl, T)
    vn = cp.copy(aristas_vl_vo_camino[0][1])
    aristas_vm_vo_camino = []
    aristas_vm_vo_camino.extend(aristas_vo_vk_camino[1:])
    aristas_vm_vo_camino.extend(aristas_vl_vo_camino)
    print('Se remueve el arco del ciclo: ', ('vo', vm))
    T.remove(('vo',aristas_vo_vk_camino[0][1]))
    T.extend([(vk, vl)])
    T_prima = T.copy()
    etiquetas[(vk, vl)] = ('-', etiquetas[('vo', vm)][1])
    for arista in aristas_vo_vk_camino[1:]:  # actualizar etiquetas de ambos caminos
        etiqueta = etiquetas[arista]
        if etiqueta[0] == '+':
            etiquetas[arista] = ('-', etiquetas[('vo', vm)][1]-etiqueta[1])
        else:
            etiquetas[arista] = ('+', etiquetas[('vo', vm)][1]-etiqueta[1])
    for arista in aristas_vl_vo_camino:
        etiqueta = etiquetas[arista]
        if etiqueta[0] == '+':
            etiquetas[arista] = ('+', etiquetas[('vo', vm)][1]+etiqueta[1])
        else:
            etiquetas[arista] = ('-', etiquetas[('vo', vm)][1]+etiqueta[1])
    # normalizacion del arbol despues de actualizar etiquetas: STEP 4
    for inidice, arista in enumerate(aristas_vm_vo_camino):
        etiqueta = etiquetas[arista]
        if etiqueta[0] == '+' and etiqueta[1] > 0 and 'vo' not in arista:  # preguntar si hay arco fuerte en el camino
            T_prima.remove(arista)
            T_prima.append(('vo', arista[1]))
            for edge in aristas_vm_vo_camino[inidice+1:]:  # actualizar etiquetas del camino
                w_actualizado = etiquetas[edge][1]-etiquetas[arista][1]
                if etiquetas[edge][0] == '+':
                    etiquetas[edge] = ('+', w_actualizado)
                else:
                    etiquetas[edge] = ('-', w_actualizado)
    T = cp.copy(T_prima)  # asignar a T el arbol normalizado
    Y = list()  # actualizar Y
    arcos_fuertes = [arco for arco in T if arco[0] == 'vo' and etiquetas[arco][0] == '+' and etiquetas[arco][1] > 0]
    for arco in arcos_fuertes:
        obtener_rama(arco[1], T, Y)
print('los nodos a extraer son:', Y)
print('LG tomo:', time.time()-init_time)

Se remueve el arco del ciclo:  ('vo', 134)
Se remueve el arco del ciclo:  ('vo', 139)
Se remueve el arco del ciclo:  ('vo', 140)
Se remueve el arco del ciclo:  ('vo', 146)
Se remueve el arco del ciclo:  ('vo', 147)
Se remueve el arco del ciclo:  ('vo', 152)
Se remueve el arco del ciclo:  ('vo', 153)
Se remueve el arco del ciclo:  ('vo', 158)
Se remueve el arco del ciclo:  ('vo', 24)
Se remueve el arco del ciclo:  ('vo', 159)
Se remueve el arco del ciclo:  ('vo', 166)
Se remueve el arco del ciclo:  ('vo', 167)
Se remueve el arco del ciclo:  ('vo', 172)
Se remueve el arco del ciclo:  ('vo', 173)
Se remueve el arco del ciclo:  ('vo', 179)
Se remueve el arco del ciclo:  ('vo', 180)
Se remueve el arco del ciclo:  ('vo', 185)
Se remueve el arco del ciclo:  ('vo', 186)
Se remueve el arco del ciclo:  ('vo', 192)
Se remueve el arco del ciclo:  ('vo', 193)
Se remueve el arco del ciclo:  ('vo', 199)
Se remueve el arco del ciclo:  ('vo', 205)
Se remueve el arco del ciclo:  ('vo', 210)
Se remueve e

Se remueve el arco del ciclo:  ('vo', 1081)
Se remueve el arco del ciclo:  ('vo', 1082)
Se remueve el arco del ciclo:  ('vo', 1083)
Se remueve el arco del ciclo:  ('vo', 1084)
Se remueve el arco del ciclo:  ('vo', 1096)
Se remueve el arco del ciclo:  ('vo', 1098)
Se remueve el arco del ciclo:  ('vo', 1099)
Se remueve el arco del ciclo:  ('vo', 1100)
Se remueve el arco del ciclo:  ('vo', 1101)
Se remueve el arco del ciclo:  ('vo', 1102)
Se remueve el arco del ciclo:  ('vo', 1103)
Se remueve el arco del ciclo:  ('vo', 1104)
Se remueve el arco del ciclo:  ('vo', 1105)
Se remueve el arco del ciclo:  ('vo', 1117)
Se remueve el arco del ciclo:  ('vo', 1119)
Se remueve el arco del ciclo:  ('vo', 1120)
Se remueve el arco del ciclo:  ('vo', 1121)
Se remueve el arco del ciclo:  ('vo', 1122)
Se remueve el arco del ciclo:  ('vo', 1123)
Se remueve el arco del ciclo:  ('vo', 1124)
Se remueve el arco del ciclo:  ('vo', 1125)
Se remueve el arco del ciclo:  ('vo', 1126)
Se remueve el arco del ciclo:  (

Se remueve el arco del ciclo:  ('vo', 2114)
Se remueve el arco del ciclo:  ('vo', 2115)
Se remueve el arco del ciclo:  ('vo', 2116)
Se remueve el arco del ciclo:  ('vo', 2117)
Se remueve el arco del ciclo:  ('vo', 2129)
Se remueve el arco del ciclo:  ('vo', 2130)
Se remueve el arco del ciclo:  ('vo', 2131)
Se remueve el arco del ciclo:  ('vo', 2132)
Se remueve el arco del ciclo:  ('vo', 2133)
Se remueve el arco del ciclo:  ('vo', 2134)
Se remueve el arco del ciclo:  ('vo', 2135)
Se remueve el arco del ciclo:  ('vo', 2136)
Se remueve el arco del ciclo:  ('vo', 2137)
Se remueve el arco del ciclo:  ('vo', 2138)
Se remueve el arco del ciclo:  ('vo', 2139)
Se remueve el arco del ciclo:  ('vo', 2156)
Se remueve el arco del ciclo:  ('vo', 2157)
Se remueve el arco del ciclo:  ('vo', 2158)
Se remueve el arco del ciclo:  ('vo', 2159)
Se remueve el arco del ciclo:  ('vo', 2161)
Se remueve el arco del ciclo:  ('vo', 2180)
Se remueve el arco del ciclo:  ('vo', 2181)
Se remueve el arco del ciclo:  (

Se remueve el arco del ciclo:  ('vo', 3313)
Se remueve el arco del ciclo:  ('vo', 3314)
Se remueve el arco del ciclo:  ('vo', 3315)
Se remueve el arco del ciclo:  ('vo', 3316)
Se remueve el arco del ciclo:  ('vo', 3317)
Se remueve el arco del ciclo:  ('vo', 3318)
Se remueve el arco del ciclo:  ('vo', 3328)
Se remueve el arco del ciclo:  ('vo', 3335)
Se remueve el arco del ciclo:  ('vo', 3336)
Se remueve el arco del ciclo:  ('vo', 3337)
Se remueve el arco del ciclo:  ('vo', 3338)
Se remueve el arco del ciclo:  ('vo', 3339)
Se remueve el arco del ciclo:  ('vo', 3340)
Se remueve el arco del ciclo:  ('vo', 3341)
Se remueve el arco del ciclo:  ('vo', 3356)
Se remueve el arco del ciclo:  ('vo', 3357)
Se remueve el arco del ciclo:  ('vo', 1847)
Se remueve el arco del ciclo:  ('vo', 3358)
Se remueve el arco del ciclo:  ('vo', 3359)
Se remueve el arco del ciclo:  ('vo', 3360)
Se remueve el arco del ciclo:  ('vo', 3361)
Se remueve el arco del ciclo:  ('vo', 3362)
Se remueve el arco del ciclo:  (

In [51]:
beneficio_lista = [peso for i,peso in w.items() if i in Y]
sum(beneficio_lista)

26086899.0259699

In [50]:
for key,peso in w.items():
    print(key, peso)

0 -2236.7886
1 24829.116
2 34347.213
3 6743.223
4 -5339.52
5 -1035.9528
6 -2328.3744
7 44052.012
8 24538.248
9 41760.36
10 55701.243
11 21413.268
12 34000.3536
13 100319.013
14 -5339.52
15 -5339.52
16 -1070.8572
17 -3924.6234
18 17958.429
19 34902.765
20 15302.196
21 25180.605
22 36499.977
23 -1399.086
24 48479.067
25 34277.769
26 12943.7028
27 9503.622
28 5680.47075
29 7900.56495
30 -561.5594
31 -3705.8819
32 15701.499
33 9225.846
34 17367.912
35 -5287.68
36 -5235.84
37 -5215.0441
38 -1911.2046
39 15866.80875
40 19614.1545
41 6119.19
42 45336.726
43 23558.4945
44 550.1785
45 -2472.48
46 38114.55
47 28781.90775
48 7697.40245
49 -1907.1756
50 19364.67
51 17975.79
52 1690.929
53 -5272.6758
54 -5239.4558
55 -5687.7443
56 10938.05775
57 15371.64
58 -2011.8475
59 34503.462
60 10513.0935
61 16517.466
62 10049.08275
63 303.4613
64 36576.726
65 40262.9724
66 3485.13525
67 36836.1288
68 33044.3412
69 19627.7604
70 44921.58
71 26621.568
72 31096.386
73 2159.73
74 30684.042
75 15166.39275
76 4770

643 33465.6348
644 14594.05016
645 22479.7392
646 68854.4424
647 99867.9876
648 56401.8082
649 11779.01054
650 27940.3524
651 85625.1684
652 116800.8948
653 127981.2336
654 90684.60336
655 22723.2288
656 44095.3416
657 69875.2692
658 182133.81
659 211462.326
660 212758.4688
661 30476.94202
662 9930.8496
663 13114.494
664 -6343.68
665 39737.0088
666 76875.66
667 146632.158
668 170677.8648
669 169446.3912
670 201821.184
671 190138.3884
672 153923.1972
673 90811.84194
674 27856.92412
675 13463.2344
676 22909.9332
677 72314.7096
678 99576.9036
679 81558.5004
680 121419.3564
681 138757.2084
682 144039.582
683 120949.0164
684 102493.116
685 74347.4628
686 53039.7288
687 -5890.56
688 -5890.56
689 -5890.56
690 -5833.92
691 -5833.92
692 -2503.3841
693 45222.2904
694 95939.5584
695 94837.7136
696 117449.4744
697 118016.6004
698 119474.9244
699 117255.0312
700 79419.6252
701 39282.8724
702 33806.0556
703 9565.47
704 -6003.84
705 -5890.56
706 -5890.56
707 -5890.56
708 -5833.92
709 -5833.92
710 -51

In [49]:
type(w)

dict

In [47]:
print(w)

{0: -2236.7886, 1: 24829.116, 2: 34347.213, 3: 6743.223, 4: -5339.52, 5: -1035.9528, 6: -2328.3744, 7: 44052.012, 8: 24538.248, 9: 41760.36, 10: 55701.243, 11: 21413.268, 12: 34000.3536, 13: 100319.013, 14: -5339.52, 15: -5339.52, 16: -1070.8572, 17: -3924.6234, 18: 17958.429, 19: 34902.765, 20: 15302.196, 21: 25180.605, 22: 36499.977, 23: -1399.086, 24: 48479.067, 25: 34277.769, 26: 12943.7028, 27: 9503.622, 28: 5680.47075, 29: 7900.56495, 30: -561.5594, 31: -3705.8819, 32: 15701.499, 33: 9225.846, 34: 17367.912, 35: -5287.68, 36: -5235.84, 37: -5215.0441, 38: -1911.2046, 39: 15866.80875, 40: 19614.1545, 41: 6119.19, 42: 45336.726, 43: 23558.4945, 44: 550.1785, 45: -2472.48, 46: 38114.55, 47: 28781.90775, 48: 7697.40245, 49: -1907.1756, 50: 19364.67, 51: 17975.79, 52: 1690.929, 53: -5272.6758, 54: -5239.4558, 55: -5687.7443, 56: 10938.05775, 57: 15371.64, 58: -2011.8475, 59: 34503.462, 60: 10513.0935, 61: 16517.466, 62: 10049.08275, 63: 303.4613, 64: 36576.726, 65: 40262.9724, 66: 348

In [40]:
print(aristas_vo_vk_camino)

[('vo', 38), (31, 38), (36, 31)]


In [31]:
print('vm:', cp.copy(aristas_vo_vk_camino[0][1]))

vm: 38


In [32]:
print(aristas_vl_vo_camino)

[('vo', 37)]


In [33]:
print(aristas_vm_vo_camino)

[(31, 38), (36, 31), ('vo', 37)]


In [38]:
T

[('vo', 1),
 ('vo', 6),
 ('vo', 7),
 ('vo', 8),
 ('vo', 12),
 ('vo', 13),
 ('vo', 15),
 ('vo', 17),
 ('vo', 18),
 ('vo', 19),
 ('vo', 20),
 ('vo', 21),
 ('vo', 22),
 ('vo', 23),
 ('vo', 24),
 ('vo', 25),
 ('vo', 26),
 ('vo', 27),
 ('vo', 28),
 ('vo', 30),
 ('vo', 32),
 ('vo', 33),
 ('vo', 37),
 ('vo', 38),
 ('vo', 39),
 ('vo', 40),
 ('vo', 41),
 ('vo', 42),
 ('vo', 43),
 ('vo', 44),
 ('vo', 45),
 ('vo', 46),
 ('vo', 47),
 ('vo', 48),
 ('vo', 49),
 ('vo', 50),
 ('vo', 51),
 ('vo', 52),
 ('vo', 53),
 ('vo', 54),
 ('vo', 55),
 ('vo', 56),
 ('vo', 57),
 ('vo', 58),
 ('vo', 59),
 ('vo', 60),
 ('vo', 61),
 ('vo', 62),
 ('vo', 63),
 ('vo', 64),
 ('vo', 65),
 ('vo', 66),
 ('vo', 67),
 ('vo', 68),
 ('vo', 69),
 ('vo', 70),
 ('vo', 71),
 ('vo', 72),
 ('vo', 73),
 ('vo', 74),
 ('vo', 75),
 ('vo', 76),
 ('vo', 77),
 ('vo', 78),
 ('vo', 79),
 ('vo', 80),
 ('vo', 81),
 ('vo', 82),
 ('vo', 83),
 ('vo', 84),
 ('vo', 85),
 ('vo', 86),
 ('vo', 87),
 ('vo', 88),
 ('vo', 89),
 ('vo', 90),
 ('vo', 91),
 ('